In [ ]:
%cd /content/drive/MyDrive/project demo

[Errno 2] No such file or directory: '/content/drive/MyDrive/project demo'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.0/883.0 kB 19.9 MB/s eta 0:00:00


## vehicle and Lp detection and Recognition

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Import the YOLO model from the Ultralytics library for object detection and tracking
from ultralytics import YOLO
# Import the OpenCV library for video processing
import cv2

# Load the pretrained YOLO model on COCO dataset
vehicle_model = YOLO("/content/drive/MyDrive/project_demo/weights/vehicle_detection.pt")
lp_model=YOLO("/content/drive/MyDrive/project_demo/weights/vehicle_detection.pt")
LP_recognise_model=YOLO("/content/best.pt")
# Open the video file for reading using OpenCV's VideoCapture object
cap = cv2.VideoCapture("/content/drive/MyDrive/project_demo/video.webm")
# Check if the video file is opened successfully
assert cap.isOpened(), "Error reading video file"

# Retrieve the width, height, and frames per second (fps) of the video
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Create a VideoWriter object to save the output video with Object Tracking
video_writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Loop through the video frames as long as the video is open
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    # Check if the frame is read successfully
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform object tracking on the current frame using the YOLO model
    vehicle_detected = vehicle_model.predict(frame,imgsz=640,conf=0.5,classes=[0,1,2,3,4,6,7,8,9,10,11,12,13,14])

    # Visualize the results on the frame
    annotated_frame=vehicle_detected[0].plot()

    if len(vehicle_detected[0].boxes.cls.tolist()) > 0:
      for box in vehicle_detected[0].boxes:
        cls=int(box.cls[0]) # index  of predicted object
        if cls != 5:
          x1_veh, y1_veh, x2_veh, y2_veh = map(int, box.xyxy[0])
          cropped_vehicle = frame[y1_veh:y2_veh, x1_veh:x2_veh]
          LP_detected = lp_model.predict(
                    cropped_vehicle, imgsz=640, classes=[5], conf=0.5, max_det=1
                )
          if len(LP_detected[0].boxes.cls.tolist()) > 0:
            x1,y1,x2,y2=map(int,LP_detected[0].boxes.xyxy[0])
            cropped_LP=cropped_vehicle[y1:y2, x1:x2]
            gray_LP = cv2.cvtColor(cropped_LP, cv2.COLOR_BGR2GRAY)
            rgb_LP = cv2.cvtColor(gray_LP, cv2.COLOR_GRAY2BGR)
            resized_LP = cv2.resize(rgb_LP, (640, 320))
            recognized_text, recognised_img = recognize_license_plate(resized_LP)
            text = recognized_text
            font_scale = 1
            font_thickness = 2
            text_size, baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)

            # Coordinates for the text
            text_x = x1 + x1_veh
            text_y = y1 + y1_veh - 40

            cv2.rectangle(annotated_frame,(text_x, text_y - text_size[1]),(text_x + text_size[0], text_y + baseline),(255, 255, 255),cv2.FILLED,)
            # Draw the text on the rectangle (black text)
            cv2.putText(annotated_frame,text,(text_x, text_y),cv2.FONT_HERSHEY_SIMPLEX,font_scale,(0, 0, 0),font_thickness,)
            # cv2.putText(annotated_frame, recognized_text, (x1+x1_veh, y1+y1_veh - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
      # vehicle_detected=None


    # Write the annotated frame with Object IDs to the output video
    video_writer.write(annotated_frame)

# Release the video capture, video writer, and close any OpenCV windows
cap.release()
video_writer.release()
cv2.destroyAllWindows()



0: 384x640 2 bikes, 1 bus, 3 scooters, 888.8ms
Speed: 3.0ms preprocess, 888.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 1 license_plate, 1744.3ms
Speed: 5.6ms preprocess, 1744.3ms inference, 1.2ms postprocess per image at shape (1, 3, 544, 640)

0: 320x640 2 4s, 2 5s, 1 8, 1 ba, 1 ca, 711.6ms
Speed: 1.8ms preprocess, 711.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
recognised text----- ba 8 ca 4 5 5 4

0: 640x256 (no detections), 841.9ms
Speed: 5.5ms preprocess, 841.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 256)

0: 640x192 (no detections), 738.9ms
Speed: 1.4ms preprocess, 738.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 192)

0: 640x288 (no detections), 1044.4ms
Speed: 5.7ms preprocess, 1044.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 288)

0: 640x256 1 license_plate, 953.8ms
Speed: 2.0ms preprocess, 953.8ms inference, 1.4ms postprocess per image at shape (1, 3, 6

In [ ]:
def recognize_license_plate(cropped_LP):
    detections = LP_recognise_model.predict(cropped_LP, imgsz=640)

    if len(detections[0].boxes.cls.tolist()) > 0:
        boxes = []
        labels = []
        # Iterate through each detection
        for detection in detections:
            # Check if the detection has the 'boxes' attribute
            # Extract box coordinates and class indices
            boxes += detection.boxes.xyxy.tolist()  # Convert tensor to list
            class_indices = detection.boxes.cls.tolist()

            # Convert class indices to actual labels using the names dictionary
            labels += [detection.names[int(cls_idx)] for cls_idx in class_indices]

        # Combine the bounding boxes with their respective labels
        detections_with_labels = list(zip(boxes, labels))

        # Function to determine if two boxes are in the same row (y-coordinate proximity)
        def are_in_same_row(box1, box2, threshold=25): #25
            return abs(box1[1] - box2[1]) < threshold  # Compare y1 values

        # Sort detections by their y-coordinate first (top to bottom)
        detections_sorted_y = sorted(detections_with_labels, key=lambda x: x[0][1])

        current_row = (
            [detections_sorted_y[0]] if detections_sorted_y else []
        )  # Start with the first detection
        rows = []  # To store rows

        # Group detections into rows based on y1 proximity
        for i in range(1, len(detections_sorted_y)):
            current_box = detections_sorted_y[i][0]
            previous_box = current_row[-1][0]

            if are_in_same_row(current_box, previous_box):
                # If the current detection is in the same row, add it to the current row
                current_row.append(detections_sorted_y[i])
            else:
                # Otherwise, start a new row
                rows.append(current_row)
                current_row = [detections_sorted_y[i]]

        # Don't forget to add the last row
        rows.append(current_row)

        # Sort each row by x-coordinate (left to right)
        rows_sorted = [sorted(row, key=lambda x: x[0][0]) for row in rows]

        # Concatenate the labels row by row
        recognized_text = " ".join(
            [detection[1] for row in rows_sorted for detection in row]
        )
        print("recognised text-----",recognized_text)
        return recognized_text, detections[0].plot()
    else:
        return None, None

